In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

데이터 로드 - 시청률 데이터는 제 폴더 안에 있어요

In [3]:
sales= pd.read_excel('/content/sales.xlsx',header =1)
sales_test= pd.read_excel('/content/sales_test.xlsx',header =1)
daily_top_3= pd.read_excel('/content/2019_daily_top3.xlsx')
daily_top_3_test= pd.read_excel('/content/2020_daily_top3.xlsx')

ns 홈쇼핑 시청률 매칭(필요 시 사용해주세욥!)

In [ ]:
def vr_match():
  vr = list(sales['방송일시'])
  
#방송일시 type Timestamp -> 문자열로 처리
  for i in range(len(vr)):
    vr[i] = str(vr[i]).replace('Timestamp(','').replace(')','').strip()
  
#방송일시에서 날짜와 시간대를 분리하여 시간대 평균 시청률 매칭하기
  
  for i in range(len(vr)):
    try:
      date = vr[i].split(' ')[0]

      if(i<len(vr)-4): #19-12-31 23:40분 제외
        t1 = vr[i].split(' ')[1][:5] #시간
    #다음 방영 시간대 계산
        def next(t1):
          j=i
          t2 = vr[j+1].split(' ')[1][:5]
          while(t1==t2):
            j=j+1
            t2=vr[j+1].split(' ')[1][:5]
          return vr[j+1].split(' ')[1][:5] #다음 시간
        t2 = next(t1) 
      
        def count(t1,t2):
          t1= datetime.datetime.strptime(t1, '%H:%M')
          t2= datetime.datetime.strptime(t2, '%H:%M')
          if(int(str(t2-t1).split(':')[0][-1])>0):
            c=60
          else:
            c=int(str(t2-t1).split(':')[1])
          return c-1
 
  # 만약 다음 시간이 10분이면 
        if(t2[3:]=='10'): 
          time = t1[:3]+'09'
          vr[i]= view_rate[date].rolling(window=count(t1,t2)).mean()[time]
        elif(t1=='01:30'):
          vr[i]= view_rate[date].rolling(window=30).mean()['01:59']
        elif(t1=='01:40'):
          vr[i]= view_rate[date].rolling(window=20).mean()['01:59']
        elif(t1=='01:50'):
          vr[i]= view_rate[date].rolling(window=10).mean()['01:59']
  # 10분 아니면  
        else:
          hour=t1[:3] #시
          min= int(t1[3:])+count(t1,t2) 
          if(min>60): #만약 60분보다 큰 수가 나오면
            min=min-60 # 빼버리고
            if(min<10): #뺀 값이 한자리수 일 때
              min='0'+str(min)   
            new=int(hour[:2])+1# 시간은 +1
        
            if (new<10):
              new='0'+str(new)

            elif(new==24):  
              new='00'
        
            hour=str(new)+':'
      
          time = hour+str(min)
      
          vr[i]= view_rate[date].rolling(window=count(t1,t2)).mean()[time]
  
#12월 31일 11시 40분 처리
      else:
        vr[i]= view_rate[date].rolling(window=20).mean()[vr[i].split(' ')[1][:3]+'59']
    except:
      vr[i]=0
    sales['viewrate'] = vr

In [ ]:
vr_match()

전처리

In [5]:
## 1.무형 상품은 추정 제외 
## 2.결측된 노출분 채우기 (직전의 값 집어넣기) 
## 3. 판매단가 > 취급액이면 취급액 0 
##

def custom(price, total):
    # 3전처리 
    if price > total: return 0
    else: return total

def pre_processing(df): 
    # 1. 무형 상품 추정 제외 
    df = df.drop(df[df["상품군"]=="무형"].index,axis = 0)
    # 2. 결측된 노출분 채우기 
    for i,row in df.iterrows():
        val = row["노출(분)"]
        if i == 0:
            continue 
        if np.isnan(val):
            if df.loc[i,"방송일시"] == df.loc[i-1,"방송일시"]:
                df.loc[i,"노출(분)"] = df.loc[i-1,"노출(분)"]
            else :
                continue 
                
    # 3. 판매단가 > 취급액이면 취급액 0 
    df["취급액"] = df.apply(lambda x : custom(x["판매단가"], x["취급액"]),axis=1)
    print(sum((sales["판매단가"] > sales["취급액"]) & (sales['취급액'] != 0)))
    # 4. month, day, hour, minute, weekday 열 추가 
    df["방송일시"] = pd.to_datetime(df["방송일시"])
    df['month'] = pd.DatetimeIndex(df['방송일시']).month
    df['day'] = pd.DatetimeIndex(df['방송일시']).day
    df['hour'] = pd.DatetimeIndex(df['방송일시']).hour
    df['minute'] = pd.DatetimeIndex(df['방송일시']).minute
    df['weekday'] = pd.DatetimeIndex(df['방송일시']).weekday
    
    # reset_index
    df = df.reset_index(drop=True)
    
    return df 


In [7]:
sales = pre_processing(sales)


0


In [97]:
sales_test = pre_processing(sales_test)

0


외부시청률

In [ ]:
sales['viewrate1']=np.zeros((37372,1), dtype=np.int8)
sales['viewrate2']=np.zeros((37372,1), dtype=np.int8)
sales['viewrate3']=np.zeros((37372,1), dtype=np.int8)

In [39]:
date_time=[]
for info in sales['방송일시']:
  date_time.append(str(info).replace('Timestamp(','').replace(')','').strip())
time_list = pd.Series(date_time).unique()

for i in range(len(time_list)):
  time_list[i] = time_list[i].split(' ')[1][:-3]

In [14]:
date=[]
time=[]

for date_time in sales['방송일시']:
  date_time= str(date_time).replace('Timestamp(','').replace(')','').strip()
  date.append(date_time.split(' ')[0].replace('-',''))
  time.append(date_time.split(' ')[1][:-3])

sales['date']= date
sales['time']= time

In [ ]:
next= []

for i in range(len(sales)):
  try:
    def count_j():
      for j in range(len(time_list)):
        if(sales['time'][i]==time_list[j]): 
          next.append(time_list[j+1])
        return j
    j=count_j() 
     
    if(sales['time'][i]!=sales['time'][i+1]):
      time_list=time_list[j+1:]
  except:
    next.append('00:20')

sales['next_time']=next

In [67]:
def f_s(): #새벽-> 아침 시간대 조정
  for i in range(1,len(sales)):
    if (sales['next_time'][i]=='06:00' or sales['next_time'][i]=='06:20' or sales['next_time'][i]=='07:00' or sales['next_time'][i]=='07:20'):
      if(sales['next_time'][i-1][:2]=='00'): # 한 케이스 확인
        sales['next_time'][i]='01:00'
      elif(sales['next_time'][i-1][:2]=='01'):
        sales['next_time'][i]='02:00'
      elif(sales['next_time'][i-1][:2]=='02'):
        sales['next_time'][i]='02:20'
      elif(sales['next_time'][i-1][:2]=='23'): # 한 케이스 확인
        sales['next_time'][i]='00:00'


In [98]:
f_s()

In [46]:
daily_top_3

,Unnamed: 0,view_rate,date,start time,finish time,rank
0,비켜라 운명아,14.40,20190101,20:30,21:00,1
1,특선영화,12.50,20190101,19:30,22:00,2
2,특선영화,11.20,20190101,19:30,22:00,3
3,비켜라 운명아,17.80,20190102,20:30,21:00,1
4,황후의 품격,14.35,20190102,22:00,23:10,2
...,...,...,...,...,...,...
1132,꽃길만 걸어요,17.40,20191230,20:30,21:00,2
1133,KBS 뉴스 9,10.70,20191230,21:00,22:00,3
1134,우아한 모녀,10.80,20191231,19:50,20:25,1
1135,꽃길만 걸어요,15.80,20191231,20:30,21:00,2


In [ ]:
for i in range(len(sales)):
  for j in range(len(daily_top_3)):
    if(sales['date'][i]==str(daily_top_3['date'][j])):
      if(datetime.datetime.strptime(sales['time'][i],'%H:%M')<= datetime.datetime.strptime(daily_top_3['start time'][j],'%H:%M') <datetime.datetime.strptime(sales['next_time'][i],'%H:%M')):
        print(daily_top_3['date'][j],daily_top_3['start time'][j])
        print('viewrate{}'.format(str(daily_top_3['rank'][j])))
        sales['viewrate{}'.format(str(daily_top_3['rank'][j]))][i]=1
      elif(datetime.datetime.strptime(sales['time'][i],'%H:%M')< datetime.datetime.strptime(daily_top_3['finish time'][j],'%H:%M') <=datetime.datetime.strptime(sales['next_time'][i],'%H:%M')):
        print(daily_top_3['date'][j],daily_top_3['finish time'][j])
        print('viewrate{}'.format(str(daily_top_3['rank'][j])))
        sales['viewrate{}'.format(str(daily_top_3['rank'][j]))][i]=1



In [95]:
sales.to_csv('sales_viewrate.csv',encoding='euc-kr')

2020 6월

In [104]:
sales_test['viewrate1']=np.zeros((2716,1), dtype=np.int8)
sales_test['viewrate2']=np.zeros((2716,1), dtype=np.int8)
sales_test['viewrate3']=np.zeros((2716,1), dtype=np.int8)

In [105]:
date_time=[]
for info in sales_test['방송일시']:
  date_time.append(str(info).replace('Timestamp(','').replace(')','').strip())
time_list = pd.Series(date_time).unique()

for i in range(len(time_list)):
  time_list[i] = time_list[i].split(' ')[1][:-3]

In [106]:
date=[]
time=[]

for date_time in sales_test['방송일시']:
  date_time= str(date_time).replace('Timestamp(','').replace(')','').strip()
  date.append(date_time.split(' ')[0].replace('-',''))
  time.append(date_time.split(' ')[1][:-3])

sales_test['date']= date
sales_test['time']= time

In [107]:
next= []

for i in range(len(sales_test)):
  try:
    def count_j():
      for j in range(len(time_list)):
        if(sales_test['time'][i]==time_list[j]): 
          next.append(time_list[j+1])
        return j
    j=count_j() 
     
    if(sales_test['time'][i]!=sales_test['time'][i+1]):
      time_list=time_list[j+1:]
  except:
    next.append('00:20')

sales_test['next_time']=next

In [109]:
sales_test.to_csv('ss.csv',encoding='euc-kr')

In [115]:
def f_s(): #새벽-> 아침 시간대 조정
  for i in range(1,len(sales_test)):
    if (sales_test['next_time'][i]=='06:00' or sales_test['next_time'][i]=='06:20' or sales_test['next_time'][i]=='07:00' or sales_test['next_time'][i]=='07:20'):
      if(sales_test['next_time'][i-1][:2]=='00'): # 한 케이스 확인
        sales_test['next_time'][i]='01:00'
      elif(sales_test['next_time'][i-1][:2]=='01'):
        sales_test['next_time'][i]='02:00'
      elif(sales_test['next_time'][i-1][:2]=='02'):
        sales_test['next_time'][i]='02:20'
      elif(sales_test['next_time'][i-1][:2]=='23'): # 한 케이스 확인
        sales_test['next_time'][i]='00:00'


In [116]:
f_s()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [119]:
for i in range(len(sales_test)):
  for j in range(len(daily_top_3_test)):
    if(sales_test['date'][i]==str(daily_top_3_test['date'][j])):
      if(datetime.datetime.strptime(sales_test['time'][i],'%H:%M')<= datetime.datetime.strptime(daily_top_3_test['start time'][j],'%H:%M') <datetime.datetime.strptime(sales_test['next_time'][i],'%H:%M')):
        print(daily_top_3_test['date'][j],daily_top_3_test['start time'][j])
        print('viewrate{}'.format(str(daily_top_3_test['rank'][j])))
        sales_test['viewrate{}'.format(str(daily_top_3_test['rank'][j]))][i]=1
      elif(datetime.datetime.strptime(sales_test['time'][i],'%H:%M')< datetime.datetime.strptime(daily_top_3_test['finish time'][j],'%H:%M') <=datetime.datetime.strptime(sales_test['next_time'][i],'%H:%M')):
        print(daily_top_3_test['date'][j],daily_top_3_test['finish time'][j])
        print('viewrate{}'.format(str(daily_top_3_test['rank'][j])))
        sales_test['viewrate{}'.format(str(daily_top_3_test['rank'][j]))][i]=1



20200601 19:50
viewrate2
20200601 20:30
viewrate1
20200601 20:30
viewrate2
20200601 21:00
viewrate1
20200601 21:00
viewrate3
20200601 22:00
viewrate3
20200601 22:00
viewrate3
20200601 22:00
viewrate3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


20200602 19:50
viewrate2
20200602 19:50
viewrate2
20200602 19:50
viewrate2
20200602 19:50
viewrate2
20200602 19:50
viewrate2
20200602 20:30
viewrate1
20200602 20:30
viewrate2
20200602 21:00
viewrate1
20200602 21:00
viewrate3
20200602 22:00
viewrate3
20200602 22:00
viewrate3
20200603 20:30
viewrate1
20200603 21:00
viewrate1
20200603 21:00
viewrate3
20200603 22:00
viewrate3
20200603 22:00
viewrate2
20200604 00:30
viewrate2
20200604 19:50
viewrate3
20200604 19:50
viewrate3
20200604 20:30
viewrate1
20200604 20:30
viewrate3
20200604 21:00
viewrate1
20200604 22:00
viewrate2
20200605 00:00
viewrate2
20200605 20:30
viewrate1
20200605 21:00
viewrate1
20200605 21:00
viewrate3
20200605 22:00
viewrate3
20200606 00:50
viewrate2
20200606 17:30
viewrate1
20200606 18:30
viewrate1
20200606 19:55
viewrate3
20200606 21:15
viewrate2
20200606 21:15
viewrate3
20200606 21:15
viewrate2
20200606 21:15
viewrate3
20200606 21:15
viewrate2
20200606 21:15
viewrate3
20200606 21:15
viewrate2
20200606 21:15
viewrate3


In [120]:
sales_test.to_csv('sales_test_viewrate.csv',encoding='euc-kr')

함수

In [10]:
def nilsen_view_rate(sales,daily_top_3):
  sales['viewrate1']=np.zeros((37372,1), dtype=np.int8)
  sales['viewrate2']=np.zeros((37372,1), dtype=np.int8)
  sales['viewrate3']=np.zeros((37372,1), dtype=np.int8)

#
  date_time=[]
  for info in sales['방송일시']:
    date_time.append(str(info).replace('Timestamp(','').replace(')','').strip())
  time_list = pd.Series(date_time).unique()

  for i in range(len(time_list)):
    time_list[i] = time_list[i].split(' ')[1][:-3]

#
  date=[]
  time=[]

  for date_time in sales['방송일시']:
    date_time= str(date_time).replace('Timestamp(','').replace(')','').strip()
    date.append(date_time.split(' ')[0].replace('-',''))
    time.append(date_time.split(' ')[1][:-3])

  sales['date']= date
  sales['time']= time

  #
  next= []

  for i in range(len(sales)):
    try:
      def count_j():
        for j in range(len(time_list)):
          if(sales['time'][i]==time_list[j]): 
            next.append(time_list[j+1])
          return j
      j=count_j() 
     
      if(sales['time'][i]!=sales['time'][i+1]):
        time_list=time_list[j+1:]
    except:
      next.append('00:20')

  sales['next_time']=next
   #
  def f_s(): #새벽-> 아침 시간대 조정
    for i in range(1,len(sales)):
      if (sales['next_time'][i]=='06:00' or sales['next_time'][i]=='06:20' or sales['next_time'][i]=='07:00' or sales['next_time'][i]=='07:20'):
        if(sales['next_time'][i-1][:2]=='00'): # 한 케이스 확인
          sales['next_time'][i]='01:00'
        elif(sales['next_time'][i-1][:2]=='01'):
          sales['next_time'][i]='02:00'
        elif(sales['next_time'][i-1][:2]=='02'):
          sales['next_time'][i]='02:20'
        elif(sales['next_time'][i-1][:2]=='23'): # 한 케이스 확인
          sales['next_time'][i]='00:00'

    f_s()

     #
     for i in range(len(sales)):
      for j in range(len(daily_top_3)):
        if(sales['date'][i]==str(daily_top_3['date'][j])):
          if(datetime.datetime.strptime(sales['time'][i],'%H:%M')<= datetime.datetime.strptime(daily_top_3['start time'][j],'%H:%M') <datetime.datetime.strptime(sales['next_time'][i],'%H:%M')):
            print(daily_top_3['date'][j],daily_top_3['start time'][j])
            print('viewrate{}'.format(str(daily_top_3['rank'][j])))
            sales['viewrate{}'.format(str(daily_top_3['rank'][j]))][i]=1
          elif(datetime.datetime.strptime(sales['time'][i],'%H:%M')< datetime.datetime.strptime(daily_top_3['finish time'][j],'%H:%M') <=datetime.datetime.strptime(sales['next_time'][i],'%H:%M')):
            print(daily_top_3['date'][j],daily_top_3['finish time'][j])
            print('viewrate{}'.format(str(daily_top_3['rank'][j])))
            sales['viewrate{}'.format(str(daily_top_3['rank'][j]))][i]=1
